In [1]:
#!/usr/bin/env python3
"""
Neven Caplar 
Last updated: 2023-10-07

Goals: 
Fit the data

Each Section can/should run independently,
only these initial imports should be shared among all sections

Questions:
What determines memory limit of the workers
How to partition the dataframe in order to get more workers active


"""
import numpy as np
import pandas as pd
import pyarrow as pa

# from scipy.spatial import KDTree
import matplotlib.pyplot as plt

import JaxPeriodDrwFit


from tape.ensemble import Ensemble
from tape.utils import ColumnMapper


from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
import dask
# many workers
# dask.config.set(scheduler='threads') 

# does not work
# from multiprocessing.pool import ThreadPool
# dask.config.set(pool=ThreadPool(20))

# one worker
# dask.config.set(scheduler='processes')  
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
# cluster.adapt(minimum=10, maximum=40) 

In [3]:
ens = Ensemble(client = client)  # initialize an ensemble object
ens.client_info()


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 64,Total memory: 251.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39619,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 251.68 GiB
Comm: tcp://127.0.0.1:39614,Total threads: 8
Dashboard: http://127.0.0.1:35076/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:44120,


# Setup base directory for saving output files

In [5]:
username= "wbeebe"
basedir = f"/astro/users/{username}/data/"

# Tape Single Pixel - real data 

In [6]:
# if running on baldur
data_path = "/astro/store/epyc/data3/hipscat/catalogs/tape_test/"

col_map = ColumnMapper(id_col="SDSS_NAME_dr16q_constant", 
                       time_col="mjd_ztf_source",
                       flux_col="mag_ztf_source", 
                       err_col="magerr_ztf_source",
                       band_col="band_ztf_source")

ens.from_hipscat(data_path,
                 source_subdir="tape_test_sources",
                 object_subdir="tape_test_obj",
                 column_mapper=col_map,
                 additional_cols=True,
                 sync_tables=True,
                 npartitions=10
                 )

/astro/users/wbeebe/tape/jax/celerite/notebooks_lf/PeriodDrw/.conda/lib/python3.10/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/astro/users/wbeebe/tape/jax/celerite/notebooks_lf/PeriodDrw/.conda/lib/python3.10/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


Filter data

In [7]:
ens.query("band_ztf_source == 'g'", table = 'source')
ens.prune(10)
ens.query("rMeanPSFMag_ps1_otmo < 20", table = 'object')

In [8]:
# 9min, 13 sec on baldur, for 603 sources in 4 partitions
# 7min, 30 sec on baldur, for 603 sources in 4 partitions, Nov 7
# 5min, 14 sec on baldur, for 603 sources in 10 partitions, Nov 7
# 3min, 3 sec on baldur, with padding 
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()
res_tsp = ens.batch(JaxPeriodDrwFit_instance.optimize_map, 'mjd_ztf_source', "mag_ztf_source", "magerr_ztf_source",
                compute=True, meta=None, n_init=100)

2023-11-17 15:42:13.028133: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:1022] could not allocate CUDA stream for context 0x7f665814ac70: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-17 15:42:13.028166: E external/xla/xla/stream_executor/stream.cc:323] failed to allocate stream during initialization
2023-11-17 15:42:13.028181: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:13.032102: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:13.033661: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:1022] could not allocate CUDA stream for context 0x7f665814ac70: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-17 15:42:13.033695: E external/xla/xla/stream_executor/stream.cc:323] failed to allocate stream during initialization
2023-11-17 15:42:13.033721: E extern

XlaRuntimeError: INTERNAL: Failed to instantiate GPU graphs: CaptureGpuGraph failed (<no details>): INTERNAL: Failed to begin stream capture: CUDA_ERROR_INVALID_HANDLE: invalid resource handle

2023-11-17 15:42:19.339538: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:1022] could not allocate CUDA stream for context 0x7f2450115fb0: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-17 15:42:19.339586: E external/xla/xla/stream_executor/stream.cc:323] failed to allocate stream during initialization
2023-11-17 15:42:19.339603: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:19.355983: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:19.373911: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:19.387881: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:

using GPU


2023-11-17 15:42:19.947731: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:19.948022: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:1022] could not allocate CUDA stream for context 0x7f257814b5a0: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-17 15:42:19.948068: E external/xla/xla/stream_executor/stream.cc:323] failed to allocate stream during initialization
2023-11-17 15:42:19.948095: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:19.954004: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:42:19.963672: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:

...

2023-11-17 15:43:38.597550: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:38.604748: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:38.616801: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:38.628558: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:38.641963: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:38.657501: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource

using GPU


2023-11-17 15:43:42.062362: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:42.069642: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:42.083131: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:42.089386: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:42.104444: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:42.115967: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource

using GPU


2023-11-17 15:43:45.509712: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:45.556265: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:45.583133: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:45.602924: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:45.625038: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource handle
2023-11-17 15:43:45.677449: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:941] unable to add host callback: CUDA_ERROR_INVALID_HANDLE: invalid resource

In [ ]:
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()
res_tsp_drw = ens.batch(JaxPeriodDrwFit_instance.optimize_map_drw, 'mjd_ztf_source', "mag_ztf_source", "magerr_ztf_source",
                compute=True, meta=None, n_init=100)

In [ ]:
username="wbeebe"
def pack_output_to_parquet(result, columns, output_dir, output_filename, drop_cols=[]):
    """Packs output to a dataframe, written as a parquet file. The created dataframe object is returned for inspection."""
    dfs = []

    # Construct dataframes with the results for each object. This works both in the case where we only returned
    # the best result per object (a single array) and when we return a multidimesional array for all results
    for i in range(len(result)):
        obj_data = result.iloc[i]
        # Construct a series representing the index
        obj_index = pd.Series(np.full(len(obj_data), result.index[i]), name=result.index.name)
        dfs.append(pd.DataFrame(data=obj_data, columns=columns, index=obj_index))

    # Concatenate all of the dataframes
    result_df = pd.concat(dfs)

    # Drop any columns if requested.
    if drop_cols:
        result_df = result_df.drop(columns=drop_cols)

    # Write the output to the parquet file
    pa_table = pa.Table.from_pandas(result_df)
    pa_table
    pa.parquet.write_table(pa_table, f"{output_dir}/data/{output_filename}.parquet")
    return result_df

# Create columns for result of using just the drw kernel
param_cols = ['log_drw_scale', 'log_drw_amp']
init_param_cols = ["init_" + c for c in param_cols]
drw_columns = ['min_neg_log_lh', 'neg_log_lh'] + param_cols + init_param_cols

# Create columns for result of combining the drw params with periodic params
param_cols = ['log_drw_scale', 'log_drw_amp', 'log_per_scale', 'log_per_amp']
init_param_cols = ["init_" + c for c in param_cols]
combined_columns = ['min_neg_log_lh', 'neg_log_lh'] + param_cols + init_param_cols


In [ ]:
# Save output for results from just the drw kernel
drw_df = pack_output_to_parquet(res_tsp_drw, drw_columns,
                       f"/astro/users/{username}", "res_tsp_run_g_0_drw")

In [ ]:
# Save output for results from the combined drw and periodic kernel
combined_df = pack_output_to_parquet(res_tsp, combined_columns,
                       f"/astro/users/{username}", "res_tsp_run_g_0")

# Redo but save all results (full=True)

In [ ]:
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()
res_tsp_full = ens.batch(JaxPeriodDrwFit_instance.optimize_map, 'mjd_ztf_source', "mag_ztf_source", "magerr_ztf_source",
                compute=True, meta=None, n_init=100, full=True)

In [ ]:
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()
res_tsp_drw_full = ens.batch(JaxPeriodDrwFit_instance.optimize_map_drw, 'mjd_ztf_source', "mag_ztf_source", "magerr_ztf_source",
                compute=True, meta=None, n_init=100, full=True)

In [ ]:
# Save output for results from just the drw kernel
drw_df_full = pack_output_to_parquet(res_tsp_drw, drw_columns,
                       f"/astro/users/{username}", "res_tsp_run_g_0_drw_full")

In [ ]:
# Save output for results from the combined drw and periodic kernel
combined_df = pack_output_to_parquet(res_tsp, combined_columns,
                       f"/astro/users/{username}", "res_tsp_run_g_0_full")